<a href="https://colab.research.google.com/github/InnaK342/NeuralNetworks/blob/main/TitanicNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
%matplotlib inline


In [ ]:
def sigmoid(x):
  return 1 / (1 + np.exp(-x))

In [ ]:
class NeuralNetwork:

  def __init__(self, count_input, count_hidden, count_output):
    self.w1 = np.random.randn(count_hidden, count_input) * 0.01
    self.b1 = np.zeros((count_hidden, 1))
    self.w2 = np.random.randn(count_output, count_hidden) * 0.01
    self.b2 = np.zeros((count_output, 1))

  def forward_propogarion(self, X):

    Z1 = np.dot(self.w1, X) + self.b1
    A1 = np.tanh(Z1)
    Z2 = np.dot(self.w2, A1) + self.b2
    A2 = sigmoid(Z2)

    cache = {
        'Z1': Z1,
        'A1': A1,
        'Z2': Z2,
        'A2': A2
    }

    return A2, cache

  def cost(self, X, Y):
    m = X.shape[1]
    A2, cache = self.forward_propogarion(X)
    return 1 / m * - (np.sum(np.multiply(np.log(A2), Y) + np.multiply((1 - Y), np.log(1 - A2))))

  def back_propogation(self, X, Y, learning_rate=0.01):
    m = X.shape[1]
    A2, cache = self.forward_propogarion(X)
    cost = self.cost(X, Y)
    dZ2 = A2 - Y
    dW2 = 1 / m * (dZ2.dot(cache['A1'].T))
    db2 = 1 / m * np.sum(dZ2, axis=1, keepdims=True)
    dZ1 = self.w2.T.dot(dZ2) * (1 - np.power(cache['A1'], 2))
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1, axis=1, keepdims=True)
    self.w1 -= learning_rate * dW1
    self.w2 -= learning_rate * dW2
    self.b1 -= learning_rate * db1
    self.b2 -= learning_rate * db2

  def train(self, X, Y, num_iterations=100, learning_rate=0.01, print_cost=False):
    for i in range(num_iterations):
      self.back_propogation(X, Y, learning_rate)
      cost = self.cost(X, Y)

      if i % 100 == 0:
        if print_cost:
          print ("Cost after iteration %i: %f" %(i, cost))

  def predict(self, X):
    return self.forward_propogarion(X)[0] > 0.5

  def accuracy(self, y_true, y_pred):
    return float((np.dot(y_true, y_pred.T) + np.dot(1 - y_true, 1 - y_pred.T)) / float(y_true.size) * 100)


In [ ]:
data = pd.read_csv('titanic.csv')

In [ ]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
X = data.drop(columns=['Survived', 'PassengerId', 'Name', 'Ticket', 'Cabin'])
X

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,22.0,1,0,7.2500,S
1,1,female,38.0,1,0,71.2833,C
2,3,female,26.0,0,0,7.9250,S
3,1,female,35.0,1,0,53.1000,S
4,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...
886,2,male,27.0,0,0,13.0000,S
887,1,female,19.0,0,0,30.0000,S
888,3,female,NaN,1,2,23.4500,S
889,1,male,26.0,0,0,30.0000,C


In [ ]:
y = data.Survived

In [ ]:
y

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64

In [ ]:
X = pd.get_dummies(X)

In [ ]:
X.Age = X.Age.fillna(X.Age.mean())

In [ ]:
X.head()

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,3,22.0,1,0,7.2500,0,1,0,0,1
1,1,38.0,1,0,71.2833,1,0,1,0,0
2,3,26.0,0,0,7.9250,1,0,0,0,1
3,1,35.0,1,0,53.1000,1,0,0,0,1
4,3,35.0,0,0,8.0500,0,1,0,0,1


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
X_train = np.array(X_train).T

In [ ]:
y_train = np.array(y_train).reshape((1, y_train.shape[0]))

In [ ]:
input_nodes = 10
hidden_nodes = 100
output_nodes = 1
learning_rate = 0.01

In [ ]:
n = NeuralNetwork(input_nodes, hidden_nodes, output_nodes)

In [ ]:
n.train(X_train, y_train, 5000, learning_rate, True)


Cost after iteration 0: 0.396142
Cost after iteration 100: 0.396017
Cost after iteration 200: 0.395893
Cost after iteration 300: 0.395770
Cost after iteration 400: 0.395647
Cost after iteration 500: 0.395525
Cost after iteration 600: 0.395404
Cost after iteration 700: 0.395284
Cost after iteration 800: 0.395165
Cost after iteration 900: 0.395047
Cost after iteration 1000: 0.394930
Cost after iteration 1100: 0.394813
Cost after iteration 1200: 0.394697
Cost after iteration 1300: 0.394581
Cost after iteration 1400: 0.394467
Cost after iteration 1500: 0.394353
Cost after iteration 1600: 0.394241
Cost after iteration 1700: 0.394131
Cost after iteration 1800: 0.394023
Cost after iteration 1900: 0.393918
Cost after iteration 2000: 0.393818
Cost after iteration 2100: 0.393724
Cost after iteration 2200: 0.393640
Cost after iteration 2300: 0.393570
Cost after iteration 2400: 0.393521
Cost after iteration 2500: 0.393501
Cost after iteration 2600: 0.393511
Cost after iteration 2700: 0.393538
Cost

In [ ]:
X_test = np.array(X_test).T
y_test = np.array(y_test).reshape((1, y_test.shape[0]))

In [ ]:
predictions = n.predict(X_test)

In [ ]:
n.accuracy(y_test, predictions)

82.08955223880598